# Full Summary Dashboard (Finance Channel Format)

Comprehensive decomposition summary for all lenders using finance channel data:

**Year-over-Year (YoY) Analysis:**
1. Multi-Lender Summary (2x2 grid: Aggregate, By Tier, By Channel, Summary)
2. Per-Lender Volume Decomposition - Aggregate (2-column grid)
3. Per-Lender Volume Decomposition - By Channel (2-column grid)
4. Per-Lender Penetration Charts (2-column grids)

**Month-over-Month (MoM) Analysis:**
5. Multi-Lender Summary (2x2 grid)
6. Per-Lender Volume Decomposition - Aggregate (2-column grid)
7. Per-Lender Volume Decomposition - By Channel (2-column grid)
8. Per-Lender Penetration Charts (2-column grids)

All charts saved to `outputs/Summary/` and `outputs/{Lender}/` folders.

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
import math

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Import calculators
from lmdi_decomposition_calculator import (
    calculate_decomposition,
    calculate_finance_channel_decomposition,
    calculate_multi_lender_decomposition
)
from lmdi_penetration_calculator import (
    calculate_penetration_decomposition,
    print_penetration_decomposition
)

# Import visualization functions
from visualization_engine import (
    create_channel_waterfall_grid,
    create_multi_lender_waterfall_grid
)
from visualization_summary import _create_aggregate_waterfall, _create_channel_stacked_waterfall
from visualization_penetration import (
    create_penetration_chart_net_all_inclusive,
    create_penetration_chart_net_volume_inclusive,
    _create_net_all_inclusive_waterfall,
    _create_net_volume_inclusive_waterfall
)

# Import dimension config
from dimension_config import get_lender_tier, get_lender_tier_map, LENDER_TIER_MAP

print("Modules loaded successfully!")
print(f"Lender tier mapping: {LENDER_TIER_MAP}")

---
## User Configuration

**Specify the analysis date below.** The notebook will automatically:
- Calculate YoY (Year-over-Year) and MoM (Month-over-Month) comparisons
- Generate charts for ALL lenders in the dataset

In [ ]:
# ============================================================
# USER INPUT: Specify your analysis date here
# ============================================================

ANALYSIS_DATE = '2024-02-01'  # Format: 'YYYY-MM-01' (first of month)

# ============================================================

In [ ]:
# Load data (finance channel format)
data_path = Path.cwd().parent / 'data' / 'sample_channel_data.csv'
df = pd.read_csv(data_path)
df['month_begin_date'] = pd.to_datetime(df['month_begin_date'])

# Parse analysis date and calculate comparison dates
analysis_date = pd.to_datetime(ANALYSIS_DATE)
yoy_date = analysis_date - relativedelta(years=1)
mom_date = analysis_date - relativedelta(months=1)

# Get available dates and lenders
available_dates = df['month_begin_date'].unique()
all_lenders = sorted(df['lender'].unique())

print(f"Data loaded: {len(df)} rows")
print(f"Date range: {df['month_begin_date'].min().date()} to {df['month_begin_date'].max().date()}")
print(f"Lenders: {all_lenders}")
print(f"Finance channels: {df['finance_channel'].unique().tolist()}")
print()
print("=" * 60)
print(f"Analysis Date:   {analysis_date.date()}")
print(f"YoY Comparison:  {yoy_date.date()} -> {analysis_date.date()}")
print(f"MoM Comparison:  {mom_date.date()} -> {analysis_date.date()}")
print("=" * 60)

# Validate dates
if analysis_date not in available_dates:
    raise ValueError(f"Date '{analysis_date.date()}' not found in data.")
yoy_available = yoy_date in available_dates
mom_available = mom_date in available_dates
if not yoy_available:
    print(f"WARNING: YoY date {yoy_date.date()} not in data - YoY analysis will be skipped")
if not mom_available:
    print(f"WARNING: MoM date {mom_date.date()} not in data - MoM analysis will be skipped")

In [ ]:
# Create output directories
base_output_dir = Path.cwd().parent / 'outputs'
summary_dir = base_output_dir / 'Summary'
summary_dir.mkdir(parents=True, exist_ok=True)

# Create per-lender directories
lender_dirs = {}
for lender in all_lenders:
    lender_dir = base_output_dir / lender
    lender_dir.mkdir(parents=True, exist_ok=True)
    lender_dirs[lender] = lender_dir

print(f"Output directories created:")
print(f"  Summary: {summary_dir}")
for lender, ldir in lender_dirs.items():
    print(f"  {lender}: {ldir}")

---
## Helper Functions

In [ ]:
def create_lender_grid(lenders, create_chart_func, title_prefix, n_cols=2, figsize_per_chart=(14, 8)):
    """
    Create a grid of charts for multiple lenders.
    
    Args:
        lenders: List of lender names
        create_chart_func: Function that takes (ax, lender) and creates a chart
        title_prefix: Prefix for the overall figure title
        n_cols: Number of columns in the grid (default 2)
        figsize_per_chart: Size of each chart in the grid
    
    Returns:
        matplotlib Figure
    """
    n_lenders = len(lenders)
    n_rows = math.ceil(n_lenders / n_cols)
    
    fig_width = figsize_per_chart[0] * n_cols
    fig_height = figsize_per_chart[1] * n_rows
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(fig_width, fig_height))
    
    # Flatten axes for easy iteration
    if n_rows == 1 and n_cols == 1:
        axes = np.array([axes])
    axes = axes.flatten() if n_rows > 1 or n_cols > 1 else axes
    
    # Create charts for each lender
    for i, lender in enumerate(lenders):
        create_chart_func(axes[i], lender)
    
    # Hide unused subplots (fill with blank space)
    for i in range(n_lenders, len(axes)):
        axes[i].axis('off')
    
    fig.suptitle(title_prefix, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    
    return fig


def save_individual_charts_from_grid(fig, output_dir, prefix, chart_names):
    """
    Extract and save individual charts from a grid figure.
    
    Args:
        fig: matplotlib Figure with subplots
        output_dir: Directory to save charts
        prefix: Filename prefix (e.g., 'yoy_multi_lender')
        chart_names: List of chart names corresponding to each subplot
    """
    axes = fig.get_axes()
    for i, (ax, name) in enumerate(zip(axes, chart_names)):
        if name:  # Skip if name is None
            # Create a new figure for this chart
            extent = ax.get_tightbbox(fig.canvas.get_renderer()).transformed(fig.dpi_scale_trans.inverted())
            fig.savefig(
                output_dir / f"{prefix}_{name}.png",
                dpi=150, 
                bbox_inches=extent.expanded(1.1, 1.1),
                facecolor='white'
            )

---
## Pre-calculate All Results

In [ ]:
# Pre-calculate all decomposition results
yoy_channel_results = {}  # lender -> FinanceChannelResults
mom_channel_results = {}
yoy_pen_results = {}  # lender -> PenetrationResults
mom_pen_results = {}

print("Calculating decomposition results...")
print()

# YoY calculations
if yoy_available:
    print("YoY Calculations:")
    for lender in all_lenders:
        print(f"  Processing {lender}...")
        try:
            yoy_channel_results[lender] = calculate_finance_channel_decomposition(
                df=df, date_a=str(yoy_date.date()), date_b=str(analysis_date.date()), lender=lender
            )
            yoy_pen_results[lender] = calculate_penetration_decomposition(
                df=df, date_a=str(yoy_date.date()), date_b=str(analysis_date.date()), lender=lender
            )
        except Exception as e:
            print(f"    WARNING: Failed for {lender}: {e}")

# MoM calculations
if mom_available:
    print("\nMoM Calculations:")
    for lender in all_lenders:
        print(f"  Processing {lender}...")
        try:
            mom_channel_results[lender] = calculate_finance_channel_decomposition(
                df=df, date_a=str(mom_date.date()), date_b=str(analysis_date.date()), lender=lender
            )
            mom_pen_results[lender] = calculate_penetration_decomposition(
                df=df, date_a=str(mom_date.date()), date_b=str(analysis_date.date()), lender=lender
            )
        except Exception as e:
            print(f"    WARNING: Failed for {lender}: {e}")

# Multi-lender calculations
print("\nMulti-Lender Calculations:")
multi_yoy_results = None
multi_mom_results = None

if yoy_available:
    print("  YoY Multi-Lender...")
    multi_yoy_results = calculate_multi_lender_decomposition(
        df=df, date_a=str(yoy_date.date()), date_b=str(analysis_date.date())
    )

if mom_available:
    print("  MoM Multi-Lender...")
    multi_mom_results = calculate_multi_lender_decomposition(
        df=df, date_a=str(mom_date.date()), date_b=str(analysis_date.date())
    )

print("\nAll calculations complete!")

---
# YEAR-OVER-YEAR (YoY) ANALYSIS
---

## 1. YoY Multi-Lender Summary (2x2 Grid)

In [ ]:
# YoY Multi-Lender 2x2 Grid
if multi_yoy_results:
    fig = create_multi_lender_waterfall_grid(multi_yoy_results)
    
    # Save full grid
    output_file = summary_dir / 'yoy_multi_lender_grid.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    # Save individual charts from the grid
    axes = fig.get_axes()
    chart_configs = [
        ('yoy_multi_lender_aggregate.png', 'Aggregate'),
        ('yoy_multi_lender_by_tier.png', 'By Tier'),
        ('yoy_multi_lender_by_channel.png', 'By Channel'),
        (None, 'Summary')  # Skip summary panel
    ]
    
    for ax, (filename, _) in zip(axes, chart_configs):
        if filename:
            # Create standalone figure for this chart
            fig_single, ax_single = plt.subplots(figsize=(14, 8))
            # Copy the content (simplified - just save with bbox)
            extent = ax.get_tightbbox(fig.canvas.get_renderer())
            if extent:
                extent = extent.transformed(fig.dpi_scale_trans.inverted())
                fig.savefig(
                    summary_dir / filename,
                    dpi=150,
                    bbox_inches=extent.expanded(1.15, 1.15),
                    facecolor='white'
                )
                print(f"Saved: {summary_dir / filename}")
            plt.close(fig_single)
    
    plt.show()

## 2. YoY Per-Lender Volume Decomposition - Aggregate

In [ ]:
# YoY Volume Grid - Aggregate
if yoy_available and yoy_channel_results:
    lenders_with_results = [l for l in all_lenders if l in yoy_channel_results]
    
    def create_yoy_volume_chart(ax, lender):
        results = yoy_channel_results[lender]
        meta = results.metadata
        delta = meta['period_2_total_bookings'] - meta['period_1_total_bookings']
        delta_sign = '+' if delta >= 0 else ''
        # Two-line title: Line 1 = Chart type, Line 2 = Lender + date range
        title = f"Aggregate Lender Effects\n{lender} YoY: {yoy_date.date()} → {analysis_date.date()}"
        _create_aggregate_waterfall(
            ax=ax,
            summary=results.aggregate_summary,
            period_1_bks=meta['period_1_total_bookings'],
            period_2_bks=meta['period_2_total_bookings'],
            title=title
        )
    
    fig = create_lender_grid(
        lenders=lenders_with_results,
        create_chart_func=create_yoy_volume_chart,
        title_prefix=f'YoY Booking Decomposition (Aggregate) by Lender: {yoy_date.date()} → {analysis_date.date()}',
        n_cols=2,
        figsize_per_chart=(14, 8)
    )
    
    output_file = summary_dir / 'yoy_volume_aggregate_by_lender_grid.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    # Save individual charts to lender folders
    for lender in lenders_with_results:
        fig_individual, ax = plt.subplots(figsize=(14, 8))
        create_yoy_volume_chart(ax, lender)
        plt.tight_layout()
        individual_file = lender_dirs[lender] / f'yoy_volume_aggregate.png'
        fig_individual.savefig(individual_file, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close(fig_individual)
        print(f"Saved: {individual_file}")
    
    plt.show()

## 3. YoY Per-Lender Volume Decomposition - By Channel (FF vs Non-FF)

In [ ]:
# YoY Volume Grid - By Channel (FF vs Non-FF stacked)
if yoy_available and yoy_channel_results:
    lenders_with_results = [l for l in all_lenders if l in yoy_channel_results]
    
    def create_yoy_channel_chart(ax, lender):
        results = yoy_channel_results[lender]
        meta = results.metadata
        
        # Get channel breakdown for title
        ff_delta = meta['channel_totals']['FF']['delta_bookings']
        nonff_delta = meta['channel_totals']['NON_FF']['delta_bookings']
        ff_sign = '+' if ff_delta >= 0 else ''
        nonff_sign = '+' if nonff_delta >= 0 else ''
        
        # Two-line title: Line 1 = Chart type, Line 2 = Lender + date range
        title = f"By Finance Channel (FF vs Non-FF)\n{lender} YoY: {yoy_date.date()} → {analysis_date.date()}"
        
        _create_channel_stacked_waterfall(
            ax=ax,
            channel_summaries=results.channel_summaries,
            period_1_bks=meta['period_1_total_bookings'],
            period_2_bks=meta['period_2_total_bookings'],
            channel_totals=meta.get('channel_totals', {}),
            title=title
        )
    
    fig = create_lender_grid(
        lenders=lenders_with_results,
        create_chart_func=create_yoy_channel_chart,
        title_prefix=f'YoY Booking Decomposition (By Channel) by Lender: {yoy_date.date()} → {analysis_date.date()}',
        n_cols=2,
        figsize_per_chart=(14, 8)
    )
    
    output_file = summary_dir / 'yoy_volume_by_channel_by_lender_grid.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    # Save individual charts to lender folders
    for lender in lenders_with_results:
        fig_individual, ax = plt.subplots(figsize=(14, 8))
        create_yoy_channel_chart(ax, lender)
        plt.tight_layout()
        individual_file = lender_dirs[lender] / f'yoy_volume_by_channel.png'
        fig_individual.savefig(individual_file, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close(fig_individual)
        print(f"Saved: {individual_file}")
    
    plt.show()

## 4. YoY Per-Lender Penetration - Net All Inclusive

In [ ]:
# YoY Penetration Net All Inclusive Grid
if yoy_available and yoy_pen_results:
    lenders_with_results = [l for l in all_lenders if l in yoy_pen_results]
    n_lenders = len(lenders_with_results)
    n_cols = 2
    n_rows = math.ceil(n_lenders / n_cols)
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(14*n_cols, 8*n_rows))
    axes = axes.flatten() if n_rows > 1 or n_cols > 1 else [axes]
    
    for i, lender in enumerate(lenders_with_results):
        ax = axes[i]
        pen_result = yoy_pen_results[lender]
        
        period_1_pen = pen_result.metadata['period_1_penetration'] * 100 * 100
        period_2_pen = pen_result.metadata['period_2_penetration'] * 100 * 100
        delta_bps = pen_result.metadata['delta_penetration_bps']
        
        # Two-line title: Line 1 = Chart type, Line 2 = Lender + date range
        title = f"Penetration (Net All Inclusive)\n{lender} YoY: {yoy_date.date()} → {analysis_date.date()}"
        
        _create_net_all_inclusive_waterfall(
            ax=ax,
            summary=pen_result.summary,
            period_1_pen=period_1_pen,
            period_2_pen=period_2_pen,
            title=title
        )
    
    for i in range(n_lenders, len(axes)):
        axes[i].axis('off')
    
    fig.suptitle(f'YoY Penetration (Net All Inclusive) by Lender: {yoy_date.date()} → {analysis_date.date()}', 
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    
    output_file = summary_dir / 'yoy_penetration_net_inclusive_by_lender_grid.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    # Save individual charts
    for lender in lenders_with_results:
        fig_individual = create_penetration_chart_net_all_inclusive(
            summary=yoy_pen_results[lender].summary,
            metadata=yoy_pen_results[lender].metadata,
            lender=lender
        )
        individual_file = lender_dirs[lender] / f'yoy_penetration_net_all_inclusive.png'
        fig_individual.savefig(individual_file, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close(fig_individual)
        print(f"Saved: {individual_file}")
    
    plt.show()

## 5. YoY Per-Lender Penetration - Net Volume Inclusive

In [ ]:
# YoY Penetration Net Volume Inclusive Grid
if yoy_available and yoy_pen_results:
    lenders_with_results = [l for l in all_lenders if l in yoy_pen_results]
    n_lenders = len(lenders_with_results)
    n_cols = 2
    n_rows = math.ceil(n_lenders / n_cols)
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(14*n_cols, 8*n_rows))
    axes = axes.flatten() if n_rows > 1 or n_cols > 1 else [axes]
    
    for i, lender in enumerate(lenders_with_results):
        ax = axes[i]
        pen_result = yoy_pen_results[lender]
        
        period_1_pen = pen_result.metadata['period_1_penetration'] * 100 * 100
        period_2_pen = pen_result.metadata['period_2_penetration'] * 100 * 100
        delta_bps = pen_result.metadata['delta_penetration_bps']
        
        # Two-line title: Line 1 = Chart type, Line 2 = Lender + date range
        title = f"Penetration (Net Volume Inclusive)\n{lender} YoY: {yoy_date.date()} → {analysis_date.date()}"
        
        _create_net_volume_inclusive_waterfall(
            ax=ax,
            summary=pen_result.summary,
            metadata=pen_result.metadata,
            period_1_pen=period_1_pen,
            period_2_pen=period_2_pen,
            title=title
        )
    
    for i in range(n_lenders, len(axes)):
        axes[i].axis('off')
    
    fig.suptitle(f'YoY Penetration (Net Volume Inclusive) by Lender: {yoy_date.date()} → {analysis_date.date()}', 
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    
    output_file = summary_dir / 'yoy_penetration_net_volume_by_lender_grid.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    # Save individual charts
    for lender in lenders_with_results:
        fig_individual = create_penetration_chart_net_volume_inclusive(
            summary=yoy_pen_results[lender].summary,
            metadata=yoy_pen_results[lender].metadata,
            lender=lender
        )
        individual_file = lender_dirs[lender] / f'yoy_penetration_net_volume_inclusive.png'
        fig_individual.savefig(individual_file, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close(fig_individual)
        print(f"Saved: {individual_file}")
    
    plt.show()

---
# MONTH-OVER-MONTH (MoM) ANALYSIS
---

## 6. MoM Multi-Lender Summary (2x2 Grid)

In [ ]:
# MoM Multi-Lender 2x2 Grid
if multi_mom_results:
    fig = create_multi_lender_waterfall_grid(multi_mom_results)
    
    # Save full grid
    output_file = summary_dir / 'mom_multi_lender_grid.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    # Save individual charts from the grid
    axes = fig.get_axes()
    chart_configs = [
        ('mom_multi_lender_aggregate.png', 'Aggregate'),
        ('mom_multi_lender_by_tier.png', 'By Tier'),
        ('mom_multi_lender_by_channel.png', 'By Channel'),
        (None, 'Summary')  # Skip summary panel
    ]
    
    for ax, (filename, _) in zip(axes, chart_configs):
        if filename:
            extent = ax.get_tightbbox(fig.canvas.get_renderer())
            if extent:
                extent = extent.transformed(fig.dpi_scale_trans.inverted())
                fig.savefig(
                    summary_dir / filename,
                    dpi=150,
                    bbox_inches=extent.expanded(1.15, 1.15),
                    facecolor='white'
                )
                print(f"Saved: {summary_dir / filename}")
    
    plt.show()

## 7. MoM Per-Lender Volume Decomposition - Aggregate

In [ ]:
# MoM Volume Grid - Aggregate
if mom_available and mom_channel_results:
    lenders_with_results = [l for l in all_lenders if l in mom_channel_results]
    
    def create_mom_volume_chart(ax, lender):
        results = mom_channel_results[lender]
        meta = results.metadata
        delta = meta['period_2_total_bookings'] - meta['period_1_total_bookings']
        delta_sign = '+' if delta >= 0 else ''
        # Two-line title: Line 1 = Chart type, Line 2 = Lender + date range
        title = f"Aggregate Lender Effects\n{lender} MoM: {mom_date.date()} → {analysis_date.date()}"
        _create_aggregate_waterfall(
            ax=ax,
            summary=results.aggregate_summary,
            period_1_bks=meta['period_1_total_bookings'],
            period_2_bks=meta['period_2_total_bookings'],
            title=title
        )
    
    fig = create_lender_grid(
        lenders=lenders_with_results,
        create_chart_func=create_mom_volume_chart,
        title_prefix=f'MoM Booking Decomposition (Aggregate) by Lender: {mom_date.date()} → {analysis_date.date()}',
        n_cols=2,
        figsize_per_chart=(14, 8)
    )
    
    output_file = summary_dir / 'mom_volume_aggregate_by_lender_grid.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    # Save individual charts to lender folders
    for lender in lenders_with_results:
        fig_individual, ax = plt.subplots(figsize=(14, 8))
        create_mom_volume_chart(ax, lender)
        plt.tight_layout()
        individual_file = lender_dirs[lender] / f'mom_volume_aggregate.png'
        fig_individual.savefig(individual_file, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close(fig_individual)
        print(f"Saved: {individual_file}")
    
    plt.show()

## 8. MoM Per-Lender Volume Decomposition - By Channel (FF vs Non-FF)

In [ ]:
# MoM Volume Grid - By Channel (FF vs Non-FF stacked)
if mom_available and mom_channel_results:
    lenders_with_results = [l for l in all_lenders if l in mom_channel_results]
    
    def create_mom_channel_chart(ax, lender):
        results = mom_channel_results[lender]
        meta = results.metadata
        
        # Get channel breakdown for title
        ff_delta = meta['channel_totals']['FF']['delta_bookings']
        nonff_delta = meta['channel_totals']['NON_FF']['delta_bookings']
        ff_sign = '+' if ff_delta >= 0 else ''
        nonff_sign = '+' if nonff_delta >= 0 else ''
        
        # Two-line title: Line 1 = Chart type, Line 2 = Lender + date range
        title = f"By Finance Channel (FF vs Non-FF)\n{lender} MoM: {mom_date.date()} → {analysis_date.date()}"
        
        _create_channel_stacked_waterfall(
            ax=ax,
            channel_summaries=results.channel_summaries,
            period_1_bks=meta['period_1_total_bookings'],
            period_2_bks=meta['period_2_total_bookings'],
            channel_totals=meta.get('channel_totals', {}),
            title=title
        )
    
    fig = create_lender_grid(
        lenders=lenders_with_results,
        create_chart_func=create_mom_channel_chart,
        title_prefix=f'MoM Booking Decomposition (By Channel) by Lender: {mom_date.date()} → {analysis_date.date()}',
        n_cols=2,
        figsize_per_chart=(14, 8)
    )
    
    output_file = summary_dir / 'mom_volume_by_channel_by_lender_grid.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    # Save individual charts to lender folders
    for lender in lenders_with_results:
        fig_individual, ax = plt.subplots(figsize=(14, 8))
        create_mom_channel_chart(ax, lender)
        plt.tight_layout()
        individual_file = lender_dirs[lender] / f'mom_volume_by_channel.png'
        fig_individual.savefig(individual_file, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close(fig_individual)
        print(f"Saved: {individual_file}")
    
    plt.show()

## 9. MoM Per-Lender Penetration - Net All Inclusive

In [ ]:
# MoM Penetration Net All Inclusive Grid
if mom_available and mom_pen_results:
    lenders_with_results = [l for l in all_lenders if l in mom_pen_results]
    n_lenders = len(lenders_with_results)
    n_cols = 2
    n_rows = math.ceil(n_lenders / n_cols)
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(14*n_cols, 8*n_rows))
    axes = axes.flatten() if n_rows > 1 or n_cols > 1 else [axes]
    
    for i, lender in enumerate(lenders_with_results):
        ax = axes[i]
        pen_result = mom_pen_results[lender]
        
        period_1_pen = pen_result.metadata['period_1_penetration'] * 100 * 100
        period_2_pen = pen_result.metadata['period_2_penetration'] * 100 * 100
        delta_bps = pen_result.metadata['delta_penetration_bps']
        
        # Two-line title: Line 1 = Chart type, Line 2 = Lender + date range
        title = f"Penetration (Net All Inclusive)\n{lender} MoM: {mom_date.date()} → {analysis_date.date()}"
        
        _create_net_all_inclusive_waterfall(
            ax=ax,
            summary=pen_result.summary,
            period_1_pen=period_1_pen,
            period_2_pen=period_2_pen,
            title=title
        )
    
    for i in range(n_lenders, len(axes)):
        axes[i].axis('off')
    
    fig.suptitle(f'MoM Penetration (Net All Inclusive) by Lender: {mom_date.date()} → {analysis_date.date()}', 
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    
    output_file = summary_dir / 'mom_penetration_net_inclusive_by_lender_grid.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    # Save individual charts
    for lender in lenders_with_results:
        fig_individual = create_penetration_chart_net_all_inclusive(
            summary=mom_pen_results[lender].summary,
            metadata=mom_pen_results[lender].metadata,
            lender=lender
        )
        individual_file = lender_dirs[lender] / f'mom_penetration_net_all_inclusive.png'
        fig_individual.savefig(individual_file, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close(fig_individual)
        print(f"Saved: {individual_file}")
    
    plt.show()

## 10. MoM Per-Lender Penetration - Net Volume Inclusive

In [ ]:
# MoM Penetration Net Volume Inclusive Grid
if mom_available and mom_pen_results:
    lenders_with_results = [l for l in all_lenders if l in mom_pen_results]
    n_lenders = len(lenders_with_results)
    n_cols = 2
    n_rows = math.ceil(n_lenders / n_cols)
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(14*n_cols, 8*n_rows))
    axes = axes.flatten() if n_rows > 1 or n_cols > 1 else [axes]
    
    for i, lender in enumerate(lenders_with_results):
        ax = axes[i]
        pen_result = mom_pen_results[lender]
        
        period_1_pen = pen_result.metadata['period_1_penetration'] * 100 * 100
        period_2_pen = pen_result.metadata['period_2_penetration'] * 100 * 100
        delta_bps = pen_result.metadata['delta_penetration_bps']
        
        # Two-line title: Line 1 = Chart type, Line 2 = Lender + date range
        title = f"Penetration (Net Volume Inclusive)\n{lender} MoM: {mom_date.date()} → {analysis_date.date()}"
        
        _create_net_volume_inclusive_waterfall(
            ax=ax,
            summary=pen_result.summary,
            metadata=pen_result.metadata,
            period_1_pen=period_1_pen,
            period_2_pen=period_2_pen,
            title=title
        )
    
    for i in range(n_lenders, len(axes)):
        axes[i].axis('off')
    
    fig.suptitle(f'MoM Penetration (Net Volume Inclusive) by Lender: {mom_date.date()} → {analysis_date.date()}', 
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    
    output_file = summary_dir / 'mom_penetration_net_volume_by_lender_grid.png'
    fig.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
    print(f"Saved: {output_file}")
    
    # Save individual charts
    for lender in lenders_with_results:
        fig_individual = create_penetration_chart_net_volume_inclusive(
            summary=mom_pen_results[lender].summary,
            metadata=mom_pen_results[lender].metadata,
            lender=lender
        )
        individual_file = lender_dirs[lender] / f'mom_penetration_net_volume_inclusive.png'
        fig_individual.savefig(individual_file, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close(fig_individual)
        print(f"Saved: {individual_file}")
    
    plt.show()

---
## Summary of Saved Files

In [ ]:
# List all saved files
print("\n" + "=" * 80)
print("SAVED FILES SUMMARY")
print("=" * 80)

# Summary folder
print(f"\n[Summary Folder: {summary_dir}]")
print("-" * 60)
for f in sorted(summary_dir.glob('*.png')):
    size_kb = f.stat().st_size / 1024
    print(f"  {f.name:<55} ({size_kb:,.0f} KB)")

# Per-lender folders
for lender in all_lenders:
    lender_dir = lender_dirs[lender]
    files = list(lender_dir.glob('*.png'))
    if files:
        print(f"\n[{lender} Folder: {lender_dir}]")
        print("-" * 60)
        for f in sorted(files):
            size_kb = f.stat().st_size / 1024
            print(f"  {f.name:<55} ({size_kb:,.0f} KB)")

# Count totals
total_summary = len(list(summary_dir.glob('*.png')))
total_lender = sum(len(list(lender_dirs[l].glob('*.png'))) for l in all_lenders)
print("\n" + "=" * 80)
print(f"TOTAL: {total_summary} summary charts + {total_lender} lender-specific charts = {total_summary + total_lender} files")
print("=" * 80)